# 単位ベクトル化
- 手法
    - 固有名詞ベクトルを単位ベクトル化
    - 単位ベクトルに対してDBSCANでクラスタリング
    - クラスタの重心とoutlaierを取得

- 目的
    - 固有名詞のダブリを除去

In [9]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
from sklearn.cluster import DBSCAN
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim
model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [10]:
f = open('../koyuu_noun_sample_list.binaryfile','rb')
sample_noun_vec_list = pickle.load(f)

In [11]:
# 単位ベクトル化
sample_nomal_vec_list = []

for v in sample_noun_vec_list:
    a = np.array(v)
    x = np.linalg.norm(a)
    b = a / x
    sample_nomal_vec_list.append(b)

In [12]:
len(sample_nomal_vec_list)

43349

In [13]:
f = open("../0_sample_nomal_vec_list.binalyfile",'wb')
pickle.dump(sample_nomal_vec_list,f)
f.close

<function BufferedWriter.close>

In [14]:
f = open('../0_sample_nomal_vec_list.binalyfile','rb')
sample_noun_vec_list = pickle.load(f)
len(sample_noun_vec_list)

43349

In [15]:
# 密なクラスタを算出

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(sample_noun_vec_list))

for eps in range(1,10,1):
    eps = eps/10
    
    for minPts in range(2,12, 2):
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(sample_noun_vec_list)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +1)
        
        print("                            ")
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
        
    # DBSCANの結果を保存
    f_name = str(eps) + "_" + str(minPts) + "_dic.binaryfile"
    save_at = "../dbscan_koyuu_nomal_noun_fold/" + f_name
    print(save_at)
    f = open(save_at,'wb')
    pickle.dump(eps_minPts,f)
    f.close
    
    eps_minPts = {}
    minPts_data = {}
    break

noun_vec_list数 :  43349
y_dbscan [-1 -1 -1 ... -1 -1 -1]
eps: 0.1 ,minPts: 2
outlier数 43349
クラスタ数 0
                            
y_dbscan [-1 -1 -1 ... -1 -1 -1]
eps: 0.1 ,minPts: 4
outlier数 43349
クラスタ数 0
                            
y_dbscan [-1 -1 -1 ... -1 -1 -1]
eps: 0.1 ,minPts: 6
outlier数 43349
クラスタ数 0
                            
y_dbscan [-1 -1 -1 ... -1 -1 -1]
eps: 0.1 ,minPts: 8
outlier数 43349
クラスタ数 0
                            
y_dbscan [-1 -1 -1 ... -1 -1 -1]
eps: 0.1 ,minPts: 10
outlier数 43349
クラスタ数 0
                            
../dbscan_koyuu_nomal_noun_fold/0.1_10_dic.binaryfile


In [21]:
import os
import pickle
import numpy as np

path = "../dbscan_koyuu_nomal_noun_fold"
files = os.listdir(path)
files

['0.7_10_dic.binaryfile',
 '0.2_10_dic.binaryfile',
 '0.6_10_dic.binaryfile',
 '0.3_10_dic.binaryfile',
 '0.5_10_dic.binaryfile',
 '0.4_10_dic.binaryfile',
 '0.1_10_dic.binaryfile',
 '0.8_10_dic.binaryfile']

In [22]:
# データ取得
#eps minPts n_class

for file in files:
    file = "../dbscan_koyuu_nomal_noun_fold/" + file
    try:
        f = open(file,'rb')
        eps_minPts_dic = pickle.load(f)
    
        for eps, minPts_data in eps_minPts_dic.items():
            for minPts, y_dbscan in minPts_data.items():
            
                n_class = np.max(y_dbscan) +1
                n_outlier = len(np.where(y_dbscan == -1)[0])
                
                print(str(eps) +" "+ str(minPts) +" "+ str(n_class) +" " + str(n_outlier))
            
                #クラスタに含まれる点の数
                class_num = 0
                for n in range(n_class+1):
                    class_num +=  len(np.where(y_dbscan == n)[0])
    except:
        print("Error")

0.7 2 135 6961
0.7 4 10 7388
0.7 6 3 7703
0.7 8 5 7922
0.7 10 5 8127
0.2 2 0 43349
0.2 4 0 43349
0.2 6 0 43349
0.2 8 0 43349
0.2 10 0 43349
0.6 2 484 19288
0.6 4 46 20568
0.6 6 31 21272
0.6 8 17 21896
0.6 10 14 22373
0.3 2 51 43198
0.3 4 4 43310
0.3 6 1 43325
0.3 8 1 43325
0.3 10 1 43327
0.5 2 643 33999
0.5 4 104 35525
0.5 6 58 36312
0.5 8 41 36845
0.5 10 28 37302
0.4 2 296 41607
0.4 4 63 42197
0.4 6 30 42467
0.4 8 24 42627
0.4 10 18 42737
0.1 2 0 43349
0.1 4 0 43349
0.1 6 0 43349
0.1 8 0 43349
0.1 10 0 43349
Error
